In [122]:
# Import required libraries
import pandas as pd
import numpy as np

In [123]:
# Load and explore dataset
df = pd.read_csv('bank_marketing.csv')

In [124]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [126]:
# Split dataset as required to build relational database
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']].copy()
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'day','month']].copy()
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']].copy()

In [127]:
# Clean client dataframe
client['job'] = client['job'].str.replace(".", "_")

client['education'] = client['education'].str.replace(".", "_")
client['education'] = client['education'].replace("unknown", np.NaN)

for col in ['credit_default', 'mortgage']:
            client[col] = client[col].map({'yes': True, 'no': False, 'unknown': True})

In [128]:
# Clean campaign dataframe
campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent', 'failure')
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

month_to_num = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

campaign['month'] = campaign['month'].map(month_to_num)

campaign['year'] = 2022

campaign['day'] = campaign['day'].astype(str)
campaign['month'] = campaign['month'].astype(str)
campaign['year'] = campaign['year'].astype(str)

campaign['last_contact_date'] = pd.to_datetime(campaign['year'] + '-' + campaign['month'].str.zfill(2) + '-' + campaign['day'].str.zfill(2))

campaign.drop(columns=["month", "day", "year"], inplace=True)

In [129]:
# Saving dataframes to csv, without an index
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)